<img src="Swisens_logo.png" width="240" height="240" align="left"/>
<div style="text-align: right">
    SwisensDataAnalyzer Introduction
    <br>Machine Learning Model Training
    <br>Author: <a href="mailto:yanick.zeder@swisens.ch">Yanick Zeder</a>
    <br> Copyright 2021, Swisens AG
    <br> <a href="mailto:yanick.zeder@swisens.ch"> Support </a>
</div>
Adapted and modified by MeteoSwiss.

# SwisensDataAnalyzer introduction - train ml models
This notebook is a short introduction on how to train a machine learning model on the data stored in a Swisens database.

## Prerequisites
The following modules must be installed in your python environment:
- `poleno_db_interface`
- `poleno_ml`

## Required knowledge
- How to use the SwisensDataExplorer (especially how to create and work with datasets)

## Content

### Topics
This notebook takes you along the ride to train a ml model on datasets on the Swisens Database. First, we will see how we can set up a training pipeline. Afterwards, the pipeline is used to train a basic ML model. Then, we check the model on the test data and evaluate using a confusion matrix. Finally, we apply the model to a time series from a SwisensPoleno to evaluate the real-world performance of the model.

### Table of contents
1. [Add your DB credentials and connect](#query)
2. [Defining the datasets to use](#datasets)
3. [Setup the database connection and the dataset pipeline](#pipeline)
4. [Build the model](#model)
5. [Model evaluation](#evaluate)

### Imports

In [1]:
import os, sys
sys.path.append(
    "/tf/tmp/poleno-ml"
)
sys.path.append(
    "/tf/tmp/poleno-db-interface/"
)

In [2]:
import tensorflow as tf

for dev in tf.config.list_physical_devices():
    try:
        tf.config.experimental.set_memory_growth(
            dev, True
        )
        print(f"Success for {dev}")
    except:
        print(f"Failed for {dev}")

# specifies which PhysicalDevice objects are visible to the runtime. TF will only allocate memory and place operations on visible physical devices
gpu0 = tf.config.list_physical_devices('GPU')[0]
tf.config.set_visible_devices(gpu0, 'GPU')
tf.config.experimental.set_virtual_device_configuration(
    gpu0, 
    [tf.config.experimental.VirtualDeviceConfiguration(memory_limit=25_000)]
) 

# Import all other necessary modules
import datetime
import numpy as np
import os
import uuid

import tensorflow.keras as keras
from sqlalchemy import func
%load_ext autoreload
%autoreload 2
from poleno_ml.database.query_interface_ml import QueryInterfaceML
from poleno_ml.database.query_interface_ml import DatasetPipeline
import operator
import poleno_db_interface.database.model.data_explorer_model as dem
import poleno_db_interface.database.model.poleno_data_model as pdm
from uuid import UUID
import tqdm

Failed for PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')
Success for PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')
Success for PhysicalDevice(name='/physical_device:GPU:1', device_type='GPU')


### Establishing a database connection <a class="anchor" id="query"></a>

In [3]:
import myloginpath
db_config = myloginpath.parse('client', path='/tf/.mylogin.cnf')
#db_config['database'] = 'sensor_data_schema'

# Conect to the database and create an interface instance
query_interface_ml = QueryInterfaceML(**db_config)

### Create the report structure

In order to get a report at the end of the training, we first initialize the report data structure. This is simply a dictionary that will steadily be filled
data that we would like to have in our end report. The syntax is straight forward as you see below. You can also add your own data. You just have to make 
sure that you also add a placeholder for that data in the report template (TODO: Add link to jump to the report template).

In [4]:
def make_markdown_table(header, array):
    """ Create a markdown table from a list. """

    nl = "\n"

    markdown = nl
    markdown += f"| {' | '.join(header)} |"

    markdown += nl
    markdown += f"| {' | '.join(['---']*len(header))} |"

    markdown += nl
    for entry in array:
        str_array = [str(e) for e in entry]
        markdown += f"| {' | '.join(str_array)} |{nl}"

    return markdown

report_data = {}

### Defining the datasets to use <a class="anchor" id="datasets"></a>
    
<b>ATTENTION: If you change the values in here, make sure you delete all dataset cache files. These are called 'train_cache*' or 'test_cache*' and are located at the same location as this notebook. If you do not delete these caches, the training will run on the old dataset definition!!</b>

In [5]:
# A dictionary that has the form of { <collection>: { <dataset-id>: <class-label> } }
# Collection name can be choosen freely and is intended to enable grouping of datasets into logical
# units independnet on class. Typically, this can be used to seperate datasets form different systems
# but can also be used to separate different years, sample source or any other propertie. Later in the
# notebook, you will then be able to create test and validation sets with different collections as
# source.
DATASET_DEFINITIONS = {
    
    "raw-pollens": {
        "11ea8493-7107-8db4-9bf7-ae7b87f820b4": "Alnus",
        "11ea847a-f995-790c-830f-ae7b87f820b4": "Alnus",
        "11ea8475-957e-347c-985a-ae7b87f820b4": "Alnus",
        "11ea8897-f50e-66a2-9876-ae7b87f820b4": "Betula",
        "11ea8632-18ed-7210-985a-ae7b87f820b4": "Betula",
        "11ea8632-1eb2-2452-bc84-ae7b87f820b4": "Betula",
        "11ea8f77-4ee3-aef4-b330-ae7b87f820b4": "Carpinus",
        "11ea8f6d-3e75-9fe6-b46e-ae7b87f820b4": "Carpinus",
        "11ea8f6d-1562-211a-8192-ae7b87f820b4": "Carpinus",
        "11ea8f6c-b78c-d076-a542-ae7b87f820b4": "Carpinus",
        "11ea8498-b729-d4e6-bc84-ae7b87f820b4": "Corylus",
        "11ea8498-b083-cb92-a1a5-ae7b87f820b4": "Corylus",
        "11ea8498-afa9-cec4-a877-ae7b87f820b4": "Corylus",
        "11ea8fa9-6c12-723a-b3dd-ae7b87f820b4": "Cupressus",
        "11ea8fa8-fafa-aeb4-ac46-ae7b87f820b4": "Cupressus",
        "11ea8fa8-d163-dce2-b1cb-ae7b87f820b4": "Cupressus",
        "11ea8636-313b-a6e4-a69e-ae7b87f820b4": "Fagus",
        "11ea8635-ef91-6ab2-a877-ae7b87f820b4": "Fagus",
        "11ea8635-eb18-6ee0-9876-ae7b87f820b4": "Fagus",
        "11ea857e-7bc5-60a0-842e-ae7b87f820b4": "Fraxinus",
        "11ea857b-3d52-9034-830f-ae7b87f820b4": "Fraxinus",
        "11ea857b-150e-c372-bc84-ae7b87f820b4": "Fraxinus",
        "11ea8af3-c533-f39e-8b25-ae7b87f820b4": "Pinaceae",
        "11ea8af1-91fc-9a46-8b25-ae7b87f820b4": "Pinaceae",
        "11ea8af0-83dc-6d66-b06c-ae7b87f820b4": "Pinaceae",
        "11ea863d-acf6-0ade-985a-ae7b87f820b4": "Pinaceae",
        "11ea863c-2449-be52-8814-ae7b87f820b4": "Pinaceae",
        "11ea8b83-25c9-8194-90d1-ae7b87f820b4": "Platanus",
        "11ea8881-3721-9aa8-a907-ae7b87f820b4": "Platanus",
        "11ea990f-ee01-8334-b3dd-ae7b87f820b4": "Poaceae",
        "11ea990c-b2bc-fe96-b46e-ae7b87f820b4": "Poaceae",
        "11eb5fd9-961a-313e-ac56-ae7b87f820b4": "Poaceae",
        "11eb5fd9-dd36-0a20-88f3-ae7b87f820b4": "Poaceae",
        "11ebe542-660e-0206-80be-ae7b87f820b4": "Poaceae",
        "11eb5fc3-03fa-6da2-8b42-ae7b87f820b4": "Poaceae",
        "11ebe540-187e-9a0c-b0e2-ae7b87f820b4": "Poaceae",
        "11ea8893-edfb-ca84-a877-ae7b87f820b4": "Populus",
        "11ea84a0-e89b-43b8-a69e-ae7b87f820b4": "Populus",
        "11ea84a0-a2f0-ab8c-a877-ae7b87f820b4": "Populus",
        "11ea863e-1fea-0f7c-a1a5-ae7b87f820b4": "Quercus",
        "11ea863e-1b86-8226-a1a5-ae7b87f820b4": "Quercus",
        "11ea863d-f388-a038-a1a5-ae7b87f820b4": "Quercus",
        "11ea8477-cede-e7dc-897d-ae7b87f820b4": "Taxus",
        "11ea8477-b584-b690-830f-ae7b87f820b4": "Taxus",
        "11ea8494-33a5-2e4e-bc84-ae7b87f820b4": "Taxus",
        "11ea849c-df8f-d95e-897d-ae7b87f820b4": "Ulmus",
        "11ea849c-db7b-2170-8b0f-ae7b87f820b4": "Ulmus",
        "11ea849a-0e25-4018-8814-ae7b87f820b4": "Ulmus",
    },
    
    "old-pollens": {
        "11ea5df1-de4e-68a2-bdea-ae7b87f820b4": "Alnus",
        "11ea5dec-aad2-40f2-adc5-ae7b87f820b4": "Alnus",
        "11ea5dea-76dd-45e6-9881-ae7b87f820b4": "Alnus",
        "11ea8318-8f19-8414-8f2c-ae7b87f820b4": "Betula",
        "11ea831d-c087-d5fa-8016-ae7b87f820b4": "Betula",
        "11ea8319-0f9b-2f84-8f2c-ae7b87f820b4": "Betula",
        "11ea74ee-1ae2-3f42-bdc8-ae7b87f820b4": "Carpinus",
        "11ea74ef-1f33-22e0-b530-ae7b87f820b4": "Carpinus",
        "11ea74ef-7256-f794-8624-ae7b87f820b4": "Carpinus",
        "11ea74ef-cf63-afc2-bdc8-ae7b87f820b4": "Carpinus",
        "11ea5e04-fc94-364e-81ab-ae7b87f820b4": "Corylus",
        "11ea5e04-ae5b-1ace-9881-ae7b87f820b4": "Corylus",
        "11ea5e00-93c5-6f88-81ab-ae7b87f820b4": "Corylus",
        "11ea74e8-0f90-f08a-9ea9-ae7b87f820b4": "Cupressus",
        "11ea74ea-1a2b-69ec-9846-ae7b87f820b4": "Cupressus",
        "11ea74ea-93f3-4862-bc81-ae7b87f820b4": "Cupressus",
        "11ea831e-9169-688c-9d84-ae7b87f820b4": "Fagus",
        "11ea831e-5779-4480-a7e8-ae7b87f820b4": "Fagus",
        "11ea831e-0698-1618-8016-ae7b87f820b4": "Fagus",
        "11ea8314-3a9a-8644-8fc4-ae7b87f820b4": "Fraxinus",
        "11ea8313-8fef-2358-8016-ae7b87f820b4": "Fraxinus",
        "11ea8313-1742-26b2-9d84-ae7b87f820b4": "Fraxinus",
        "11ea8af1-16b3-afbe-b419-ae7b87f820b4": "Pinaceae", # Picea
        "11ea8af1-8668-a68e-b06c-ae7b87f820b4": "Pinaceae", # Picea
        "11ea8af0-7bba-7a4c-9b82-ae7b87f820b4": "Pinaceae", # Picea
        "11ea863b-0dbc-a204-8b0f-ae7b87f820b4": "Pinaceae", # Pinus
        "11ea863c-0128-16ee-a69e-ae7b87f820b4": "Pinaceae", # Pinus
        "11ea831f-8774-33e2-b44c-ae7b87f820b4": "Plantanus",
        "11ea831f-3746-b084-a59a-ae7b87f820b4": "Plantanus",
        "11ea990d-99b7-329e-b330-ae7b87f820b4": "Poaceae",
        "11ea990c-a115-87c6-b46e-ae7b87f820b4": "Poaceae",
        "11ea9a91-09a7-100e-86f2-ae7b87f820b4": "Poaceae", # Cynosurus
        "11ea9a73-0d84-81ca-b3dd-ae7b87f820b4": "Poaceae", # Cynosurus
        "11ebde5f-23d5-5e8c-8d93-ae7b87f820b4": "Poaceae", # Dactylis
        "11ea9911-17a1-65b4-89a8-ae7b87f820b4": "Poaceae", # Dactylis
        "11ebde60-40fb-4aa2-a536-ae7b87f820b4": "Poaceae", # Trisetum
        "11ea830f-df57-8bdc-91a3-ae7b87f820b4": "Populus",
        "11ea74e3-b2e6-df38-b530-ae7b87f820b4": "Populus",
        "11ea74e4-5089-a266-8624-ae7b87f820b4": "Populus",
        "11ea863a-bdae-8df4-a69e-ae7b87f820b4": "Quercus",
        "11ea863a-2252-7b54-a1a5-ae7b87f820b4": "Quercus",
        "11ea8639-acca-ee52-aa3e-ae7b87f820b4": "Quercus",
        "11ea5deb-b5f0-b38e-9881-ae7b87f820b4": "Taxus",
        "11ea5deb-6fa4-5f7a-bd51-ae7b87f820b4": "Taxus",
        "11ea5df3-6b53-0252-adc5-ae7b87f820b4": "Taxus",
        "11ea74d0-dd4c-4d34-9ea9-ae7b87f820b4": "Ulmus",
        "11ea74cf-fc64-b108-8624-ae7b87f820b4": "Ulmus",
        "11ea5ef1-f146-eabe-ab02-ae7b87f820b4": "Ulmus",
    },
    
    "new-pollens": { # same datasets as in "old-pollens" but cleaned using a different strategy
        "11ed3b18-1ba8-ee6a-a8d4-496190c661df": "Alnus",
        "11ed3821-9ab6-8dba-a8d4-496190c661df": "Alnus",
        "11ed382b-542a-0dea-a8d4-496190c661df": "Alnus",
        "11ed3832-b506-668e-a8d4-496190c661df": "Betula",
        "11ed3a58-07b4-deb6-a8d4-496190c661df": "Betula",
        "11ed38e1-b2f4-f996-a8d4-496190c661df": "Betula",
        "11ed5ea4-96ed-bdf8-acbd-a95f70cb44b0": "Carpinus",
        "11ed5f80-5dcc-9c24-acbd-a95f70cb44b0": "Carpinus",
        "11ed3a6b-5612-556c-a8d4-496190c661df": "Carpinus",
        "11ed3982-4837-efd6-a8d4-496190c661df": "Carpinus",
        "11ed38e0-6bee-47ec-a8d4-496190c661df": "Corylus",
        "11ed3a6f-f2ce-f140-a8d4-496190c661df": "Corylus",
        "11ed431c-682d-5986-a8d4-496190c661df": "Corylus",
        "11ed38ea-0973-6246-a8d4-496190c661df": "Cupressus",
        "11ed3a79-1b42-92c2-a8d4-496190c661df": "Cupressus",
        "11ed431e-c1e0-39ce-a8d4-496190c661df": "Cupressus",
        "11ed3a7c-3218-626c-a8d4-496190c661df": "Fagus",
        "11ed5f45-7fe1-e688-acbd-a95f70cb44b0": "Fagus",
        "11ed5f7b-c645-a4b2-acbd-a95f70cb44b0": "Fagus",
        "11ed3b22-8577-b13c-a8d4-496190c661df": "Fraxinus",
        "11ed5f4a-52c3-34a4-acbd-a95f70cb44b0": "Fraxinus",
        "11ed5f7e-c6f1-7f46-acbd-a95f70cb44b0": "Fraxinus",
        "11ed3d7f-b12e-ad76-a8d4-496190c661df": "Pinaceae", # Picea
        "11ed5f4d-49d0-5748-acbd-a95f70cb44b0": "Pinaceae", # Picea
        "11ed5f82-2199-44da-acbd-a95f70cb44b0": "Pinaceae", # Picea
        "11ed3d9b-7037-b6fc-a8d4-496190c661df": "Pinaceae", # Pinus
        "11ed5494-ad56-7e64-acbd-a95f70cb44b0": "Pinaceae", # Pinus
        "11ed5535-c71d-065a-acbd-a95f70cb44b0": "Pinaceae", # Pinus
        "11ed55ce-5aaf-a2ae-acbd-a95f70cb44b0": "Pinaceae", # Pinus
        "11ed55d0-cec8-bdae-acbd-a95f70cb44b0": "Pinaceae", # Pinus
        "11ed55d5-a393-64c2-acbd-a95f70cb44b0": "Pinaceae", # Pinus
        "11ed5907-4d09-f0b4-acbd-a95f70cb44b0": "Plantanus",
        "11ed6006-a52a-2448-acbd-a95f70cb44b0": "Plantanus",
        "11ed43d2-4043-b620-a8d4-496190c661df": "Poaceae",
        "11ed6009-3e1d-3184-acbd-a95f70cb44b0": "Poaceae",
        "11ed6030-3f50-655e-acbd-a95f70cb44b0": "Poaceae", # Cynosurus
        "11ed6035-488e-4564-acbd-a95f70cb44b0": "Poaceae", # Cynosurus
        "11ed6037-d194-cad4-acbd-a95f70cb44b0": "Poaceae", # Cynosurus
        "11ed6039-e772-8574-acbd-a95f70cb44b0": "Poaceae", # Cynosurus
        "11ed591a-064b-e868-acbd-a95f70cb44b0": "Poaceae", # Cynosurus
        "11ed591c-0732-bf5c-acbd-a95f70cb44b0": "Poaceae", # Cynosurus
        "11ed5922-9f70-fc56-acbd-a95f70cb44b0": "Poaceae", # Cynosurus
        "11ed5925-0949-93b6-acbd-a95f70cb44b0": "Poaceae", # Cynosurus
        "11ed5926-9845-1346-acbd-a95f70cb44b0": "Poaceae", # Cynosurus
        "11ed6032-1974-ee34-acbd-a95f70cb44b0": "Poaceae", # Dactylis
        "11ed6036-c877-51c0-acbd-a95f70cb44b0": "Poaceae", # Dactylis
        "11ed603b-8b16-bcf8-acbd-a95f70cb44b0": "Poaceae", # Dactylis
        "11ed6038-9eb8-c42a-acbd-a95f70cb44b0": "Poaceae", # Dactylis
        "11ed6010-ad76-a770-acbd-a95f70cb44b0": "Poaceae", # Dactylis
        "11ed4487-97ec-87e8-a8d4-496190c661df": "Poaceae", # Trisetum
        "11ed5928-4b40-315a-acbd-a95f70cb44b0": "Populus",
        "11ed6012-cffe-1c4a-acbd-a95f70cb44b0": "Populus",
        "11ed6041-5f45-1876-acbd-a95f70cb44b0": "Populus",
        "11ed5930-a266-322e-acbd-a95f70cb44b0": "Quercus",
        "11ed601d-4f24-ca46-acbd-a95f70cb44b0": "Quercus",
        "11ed6045-38bd-92ec-acbd-a95f70cb44b0": "Quercus",
        "11ed59c8-6d52-bbaa-acbd-a95f70cb44b0": "Taxus",
        "11ed602a-3932-49e0-acbd-a95f70cb44b0": "Taxus",
        "11ed604c-7275-f1b2-acbd-a95f70cb44b0": "Taxus",
        "11ed59ca-df2c-2188-acbd-a95f70cb44b0": "Ulmus",
        "11ed602d-6874-225c-acbd-a95f70cb44b0": "Ulmus",
        "11ed6048-9fb9-4b28-acbd-a95f70cb44b0": "Ulmus",
    },
    
    "other": {
        "11ed65b6-6a22-1968-b56b-ae7b87f820b4": "Artemisia",
        "11ed463e-f09f-6456-b550-ae7b87f820b4": "Cedrus",
        "11ec6179-fde0-042c-adac-ae7b87f820b4": "Iberulites",
        "11ec617a-fb8b-e3f2-80fb-ae7b87f820b4": "Iberulites",
        "11ec5821-b371-c3dc-8359-ae7b87f820b4": "Iberulites",
        "11ec5832-4fee-03b4-8561-ae7b87f820b4": "Iberulites",
        "11ebe542-f782-c172-bf10-ae7b87f820b4": "Waterdroplets",
        "11ebeabd-e224-d5c4-8b63-ae7b87f820b4": "Waterdroplets",
        "11ebedec-0da5-47ac-8066-ae7b87f820b4": "Waterdroplets",
        "11ebee15-1fea-4c68-9cd6-ae7b87f820b4": "Waterdroplets",
    },
    
    "spores": {
        "11ebf9db-f2e9-98cc-bc67-ae7b87f820b4": "Alternaria Solani",
        "11ec01b9-d571-ea8e-b7e1-ae7b87f820b4": "Fusarium Graminearum",
    },
}

In [6]:
model_name = 'my_first_model'
model_path = 'models'

In [7]:
batch_size = 8 # Can typically be left as is. Depending on your hardware you can try to increase the value to get better performance.
epochs = 256
model_features = [
     'rec0', 'rec1'  
] #Can typically be left as is. This defines on what data we want to train. Usually, we train on the two images 'rec0' and 'rec1'
data_filters = [
    #'blur', # remove blurry events
    #'crop', # remove cropped events
]
data_maps = [ # /!\ not fully functionnal yet !
    #'process_waves', # remove "waves" from all events
    #'holo_aug', # image augmentation
]
caching = True

collections_train = [
    #"raw-pollens",
    #"old-pollens",
    "new-pollens",
    "other",
    "spores",
]

# These two values only apply when collections_val is empty
train_part = 0.7
test_part = 0.3

# Leave empty if you'd like to train and eval on the same collections
collections_val = []

classes = set(cls 
              for collection in collections_train + collections_val 
              for cls in DATASET_DEFINITIONS[collection].values())

# Add important fields to the report
report_data["batch_size"] = batch_size
report_data["validation_type"] = "train split" if len(collections_val) == 0 else "separate dataset"
report_data["collections_train"] = collections_train
report_data["collections_val"] = collections_val
report_data["train_part"] = train_part
report_data["test_part"] = test_part
report_data["classes"] = classes

In [8]:
os.makedirs(os.path.join(model_path, model_name, "training"), exist_ok=True)
os.makedirs(os.path.join(model_path, model_name, "model"), exist_ok=True)
ds_train_cache_path = os.path.join(model_path, model_name, 'training', f'train_cache_{"_".join(collections_train)}{"_" + "_".join(data_filters + data_maps) if len(data_filters + data_maps) > 0 else ""}')
if len(collections_val) == 0:
    cache_name = f'test_cache_{"_".join(collections_train)}'
else:
    cache_name = f'test_cache_{"_".join(collections_val)}'
ds_val_cache_path = os.path.join(model_path, model_name, 'training', f'{cache_name}{"_" + "_".join(data_filters + data_maps) if len(data_filters + data_maps) > 0 else ""}')
model_timestamp = datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
checkpoint_file_path = os.path.join(model_path, model_name, 'training', 'checkpoints', model_timestamp)
logdir = os.path.join(model_path, model_name, 'training', 'logs', model_timestamp)
os.makedirs(logdir, exist_ok=True)

### Computed Values
Here we define the class weights for unbalanced datasets, the class counts, the class labels and we compute the model name.

First, we define some helper functions:

In [9]:
from typing import List
def get_dataset_mapping(ds_map: dict, collections: List[str], classes: List[str]):
    """This method filters a dataset definition for specific systems and class labels
    and returns a flat dictionary with { <dataset-id>: <class-label> } """
    
    ret = {}
    for c in collections:
        ret.update(
            { key: value for key, value in ds_map[c].items() if value in classes}
        )
    return ret

def get_dataset_sizes(ds_map_flat: dict):
    """Return a dict with <dataset-id>: <class-size>"""
    
    dataset_sizes = {}
    for k, v in dataset_map.items():
        result = query_interface_ml.session.query(
            func.count(dem.EventsInEventDataset.event_id)
        ).filter(
            dem.EventsInEventDataset.dataset_id==uuid.UUID(k).bytes
        ).all()
        dataset_sizes[k] = result[0][0]
    return dataset_sizes
    
def get_class_sizes(ds_map_flat: dict, dataset_sizes: dict):
    """Return a dict with <class-name>: <class-size>"""
    
    class_sizes = {}
    for k, v in dataset_map.items():
        size = dataset_sizes[k]
        if v not in class_sizes: class_sizes[v] = 0
        class_sizes[v] += size
    return class_sizes

def get_sorted_class_list(ds_map_flat: dict):
    return sorted(list(set(ds_map_flat.values())))

... and then we can compute the values

In [10]:
# Get the dataset collection you need for the training
dataset_map = get_dataset_mapping(
    DATASET_DEFINITIONS,
    collections=collections_train,
    classes=classes
)
assert len(dataset_map) > 0

dataset_sizes = get_dataset_sizes(dataset_map)
class_sizes = get_class_sizes(dataset_map, dataset_sizes)
classes = get_sorted_class_list(dataset_map)
num_classes = len(classes)

n_samples = sum(dataset_sizes.values())
class_counts = [class_sizes[d] for d in classes]
class_weights = n_samples / np.array(class_counts)

### Setup the dataset pipeline <a class="anchor" id="pipeline"></a>
This step builds the dataset pipeline used later to train the model. Here we have to make the decision if we would like to validate on a part of the training dataset, or if we would like to define a seperate set of datasets for validation.

If we use the same datasets for training as for validation, we have to define in what portions we would like to split the data into. Typically, we use 80% to 20% split.

#### Load train and validation sets

In [11]:
if len(collections_val) == 0:

    dataset_train, dataset_val = query_interface_ml.prepare_tf_dataset_from_poleno_datasets(
        dataset_list=list(dataset_map.keys()),
        batch_size=batch_size,
        model_features=model_features,
        labels=classes,
        dataset_label_mapping=dataset_map,
        split=(train_part, test_part)
    )

else:
    
    dataset_map_val = get_dataset_mapping(
        DATASET_DEFINITIONS,
        collections=collections_val,
        classes=classes
    )
    
    dataset_train = query_interface_ml.prepare_tf_dataset_from_poleno_datasets(
        dataset_list=list(dataset_map.keys()),
        batch_size=batch_size,
        model_features=model_features,
        labels=classes,
        dataset_label_mapping=dataset_map,
    )
    
    dataset_val = query_interface_ml.prepare_tf_dataset_from_poleno_datasets(
        dataset_list=list(dataset_map_val.keys()),
        batch_size=batch_size,
        model_features=model_features,
        labels=classes,
        dataset_label_mapping=dataset_map_val,
    )
    

#### Define then apply filters and maps (e.g. data augmentation)

In [12]:
# define data filters
# filters out images where particles are cropped
def filter_crop(rec0: tf.Tensor, rec1: tf.Tensor, T: float = .0001, BT: float = .85):
    border = [0,rec0.shape[0]-1]
    mask = np.array([
        [1. if i in border or j in border else 0. for j in range(rec0.shape[1])]
        for i in range(rec0.shape[0])
    ]).reshape(*rec0.shape)
    apply_filter_crop_ = lambda x: ((x.numpy()<BT)*mask).sum() / mask.sum() > T # return True if particle is cropped
    return not apply_filter_crop_(rec0) and not apply_filter_crop_(rec1)
apply_filter_crop = lambda ids, features, targets: tf.py_function( #py_function to work in eager mode (dataset operations are in graph mode by default)
    filter_crop, [features['rec0'], features['rec1']], Tout=tf.bool
)

# filters out images where particles are blurry
import cv2
def filter_blur(rec0: tf.Tensor, rec1: tf.Tensor, T: float = .0014):
    apply_filter_blur_ = lambda x: cv2.Laplacian(x.numpy(), cv2.CV_32F).var() < T # return True if image is blurred
    return not apply_filter_blur_(rec0) and not apply_filter_blur_(rec1)
apply_filter_blur = lambda ids, features, targets: tf.py_function( #py_function to work in eager mode (dataset operations are in graph mode by default)
    filter_blur, [features['rec0'], features['rec1']], Tout=tf.bool
)

def filter_test(rec0: tf.Tensor, rec1: tf.Tensor):
    apply_filter_test_ = lambda x: True
    return not apply_filter_test_(rec0) and not apply_filter_test_(rec1)
apply_filter_test = lambda ids, features, targets: tf.py_function( #py_function to work in eager mode (dataset operations are in graph mode by default)
    filter_test, [features['rec0'], features['rec1']], Tout=tf.bool
)

In [13]:
# define data maps
import cv2
# removes "waves" around the particles
def rmv_waves(rec: tf.Tensor):
    img = (rec.numpy()*255).astype(np.uint8)
    img = img.reshape(*img.shape[:-1])
    blurred = cv2.GaussianBlur(img, (5, 5), 0)
    _, mask = cv2.threshold(blurred, 0, 1, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    cleaned = img * ~(mask).astype(bool)
    return cleaned

# performs random image augmentation
from tensorflow.keras import Sequential
from tensorflow.keras.layers.experimental import preprocessing
holo_aug = Sequential([
	preprocessing.RandomFlip("horizontal_and_vertical"),
	preprocessing.RandomZoom(-0.05),
	preprocessing.RandomRotation(0.2)
])

In [14]:
def apply_custom_holo_map(func, *args):
    args[1]['rec0'] = tf.py_function(func, [args[1]['rec0']], Tout=[tf.float32])
    args[1]['rec1'] = tf.py_function(func, [args[1]['rec1']], Tout=[tf.float32])
    return args

In [15]:
# apply data filters and data maps
dataset_train.tf_dataset = dataset_train.tf_dataset.unbatch()
dataset_val.tf_dataset = dataset_val.tf_dataset.unbatch()

if 'blur' in data_filters:
    dataset_train.tf_dataset = dataset_train.tf_dataset.filter(apply_filter_blur)
    dataset_val.tf_dataset = dataset_val.tf_dataset.filter(apply_filter_blur)
if 'crop' in data_filters:
    dataset_train.tf_dataset = dataset_train.tf_dataset.filter(apply_filter_crop)
    dataset_val.tf_dataset = dataset_val.tf_dataset.filter(apply_filter_crop)
if 'process_waves' in data_maps:
    dataset_train.tf_dataset = dataset_train.tf_dataset.map(lambda *args: apply_custom_holo_map(rmv_waves, *args), num_parallel_calls=tf.data.AUTOTUNE, deterministic=False)
    dataset_val.tf_dataset = dataset_val.tf_dataset.map(lambda *args: apply_custom_holo_map(rmv_waves, *args), num_parallel_calls=tf.data.AUTOTUNE, deterministic=False)
if 'holo_aug' in data_maps:
    augmented_train = dataset_train.tf_dataset.map(lambda *args: apply_custom_holo_map(holo_aug, *args), num_parallel_calls=tf.data.AUTOTUNE, deterministic=False)
    dataset_train.tf_dataset = augmented_train.concatenate(dataset_train.tf_dataset)

#### Cache, shuffle, batch, and prefetch

In [16]:
# Cache the pipeline to a file in order to make subsequent training passes much faster
if caching:
    # Cache the pipeline to a file in order to make subsequent training passes much faster
    dataset_train.enable_cache(ds_train_cache_path, prepare=True)

ATTENTION: Rememeber to remove the cache file if you make changes to the dataset! Otherwies, the changes will not be reflected into the dataset and the trainingwill run on the old data.


Preparing cache:  94%|█████████▎| 137160/146672 [02:21<00:09, 966.40it/s] 


In [17]:
if caching:
    dataset_val.enable_cache(ds_val_cache_path, prepare=True)

ATTENTION: Rememeber to remove the cache file if you make changes to the dataset! Otherwies, the changes will not be reflected into the dataset and the trainingwill run on the old data.


Preparing cache:  94%|█████████▎| 58807/62864 [00:51<00:03, 1138.88it/s]


In [18]:
'%i train, %i validation' % (len(list(dataset_train.tf_dataset.as_numpy_iterator())), len(list(dataset_val.tf_dataset.as_numpy_iterator())))

'137160 train, 58807 validation'

In [19]:
dataset_train.tf_dataset = dataset_train.tf_dataset.shuffle(batch_size*100).batch(batch_size).prefetch(tf.data.AUTOTUNE)
dataset_val.tf_dataset = dataset_val.tf_dataset.shuffle(batch_size*100, reshuffle_each_iteration=False).batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [20]:
print(f"dataset lengths: (train: {dataset_train.dataset_length}, val: {dataset_val.dataset_length})")
print(f"number of classes: (train: {dataset_train.num_classes}, val: {dataset_val.num_classes})")

dataset lengths: (train: 146672, val: 62864)
number of classes: (train: 20, val: 20)


### Build the model <a class="anchor" id="model"></a>
Now we can build the model we want to train. You can use any model you can imagine as long as the model has:
- two inputs with the shape 200x200x1 with the names 'rec0' and 'rec1'
- an output layer with the name 'target' and a dimension of #classes

#### Pre-trained EffNet
This uses transferlearning in order to get fast results. You can choose between 8 models B0 - B7, each a little larger than the previous. The way this is set up is that we remove the final layer of the pretrained network and add our own output layer corresponding to our classes.

Note that in our testing, B0 is sufficient for most applications. With larger models you increase the tendency to overfit the data. If you have many classes and large datasets, you can consider increasing the model size.

In [21]:
# WARNING: THIS IS NOT A GOOD IDEA TO DO IF YOU DOWNLOAD DATA FROM UNKNOW OR UNVERIFIED SOURCES !!!
# THIS COULD INDUCE HUGE SECURITY RISKS.
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

In [22]:
inputs = []
paths = []

if "rec0" in model_features and "rec1" in model_features:
    input0 = keras.layers.Input(shape=[200,200,1], name="rec0")
    inputs.append(input0)
    input1 = keras.layers.Input(shape=[200,200,1], name="rec1")
    inputs.append(input1)

    input0_reshape = keras.layers.Concatenate()([input0,input0,input0])
    input1_reshape = keras.layers.Concatenate()([input1,input1,input1])
    input0_reshape = input0_reshape * 255 # effnet expects [0, 255] data range
    input1_reshape = input1_reshape * 255 # effnet expects [0, 255] data range

    ###                                   -> VVVV <- Change here to B0 - B7 if needed.
    effnetB0 = keras.applications.EfficientNetB0(input_shape=(200,200,3), 
                                                 include_top=False) # , weights='imagenet', drop_connect_rate=0.4
    effnetB0.trainable = False

    path0 = effnetB0(input0_reshape)
    path1 = effnetB0(input1_reshape)

    holo_path_ = keras.layers.Concatenate()([path0, path1])
    holo_path_ = keras.layers.Flatten()(holo_path_)
    
    paths.append(holo_path_)

if "fl_spectra" in model_features:
    input_fl = keras.layers.Input(shape=[13], name="fl_spectra")
    fl_path = input_fl * 255
    fl_path = keras.layers.Dense(255)(fl_path)
    inputs.append(input_fl)
    paths.append(fl_path)

if len(paths) > 1:
    path_ = keras.layers.Concatenate()(paths)
else:
    path_ = paths[0]

path_ = keras.layers.Dropout(.4)(path_)
outputs = keras.layers.Dense(
    num_classes,
    activation="sigmoid",
    name="target"
)(path_)
#outputs = keras.layers.Softmax(name="target")(outputs)


model = keras.Model(inputs=inputs, outputs=[outputs])

"done"

'done'

In [23]:
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 rec0 (InputLayer)              [(None, 200, 200, 1  0           []                               
                                )]                                                                
                                                                                                  
 rec1 (InputLayer)              [(None, 200, 200, 1  0           []                               
                                )]                                                                
                                                                                                  
 concatenate (Concatenate)      (None, 200, 200, 3)  0           ['rec0[0][0]',                   
                                                                  'rec0[0][0]',               

### Compile the model
By compiling we are adding the optimizer function and the loss function which drive the actual training. Usually, this can be left as is.

If you notice that the model trains very slowly -> increase learning_rate <br>
If the loss is fluctuating or rising -> decrease learning_rate

In [24]:
# To compile the model, we need a special loss funtion that allows for class weights. The details are not that important. Relavent to know
# is that this allows for inbalanced data to be trained correctly. For example, if you have two times more of class 1 than in class 2, then
# you would like the model to ignore this difference and act as if the two sets are identical in size.

class WeightedSCCE(tf.keras.losses.Loss):
    """Custom SparseCategoricalCrossentropy loss class that supports class weights."""
    def __init__(self, class_weight, from_logits=False, name='weighted_scce'):
        if class_weight is None or all(v == 1. for v in class_weight):
            self.class_weight = None
        else:
            self.class_weight = tf.convert_to_tensor(class_weight,
                dtype=tf.float32)
        self.reduction = keras.losses.Reduction.NONE
        self.unreduced_scce = keras.losses.SparseCategoricalCrossentropy(
            from_logits=from_logits, name=name,
            reduction=self.reduction)
        self.name = name

    def __call__(self, y_true, y_pred, sample_weight=None):
        loss = self.unreduced_scce(y_true, y_pred, sample_weight)
        if self.class_weight is not None:
            weight_mask = tf.gather(self.class_weight, y_true)
            loss = tf.math.multiply(loss, weight_mask)
        return loss

In [25]:
# Finally we compile the ml model
learning_rate = 0.000_005
model.compile(
    # Optimizer, that handles the weight adjustment while training
    optimizer=keras.optimizers.Adam(learning_rate),  
    # Loss function to minimize
    loss=WeightedSCCE(class_weights),
    # List of metrics to monitor
    metrics=[keras.metrics.SparseCategoricalAccuracy()],
)

### Setting up TensorBoard

In [26]:
# Load the TensorBoard notebook extension
%load_ext tensorboard
#%reload_ext tensorboard
# https://stackoverflow.com/questions/40106949/unable-to-open-tensorboard-in-browser

In [27]:
%tensorboard --logdir={logdir} --bind_all --port 6006

### Train the model
Here the actual training starts. The only thing to set here is the number of epochs the model should be trained.

Note, that the first epoch will take some time, as the dataset cache is built. Afterwards, the training will be only limited by your CPU/GPU performance.

In [28]:
import datetime
import io
import itertools
import sklearn.metrics
import matplotlib.pyplot as plt

In [29]:
# confusion matrix plotting at each epoch in TensorBoard
# source: https://towardsdatascience.com/exploring-confusion-matrix-evolution-on-tensorboard-e66b39f4ac12 by Surhrut Ashtikar, last visited on 28.11.2022
def plot_confusion_matrix(cm, class_names):
    """
    Returns a matplotlib figure containing the plotted confusion matrix.
    
    Args:
       cm (array, shape = [n, n]): a confusion matrix of integer classes
       class_names (array, shape = [n]): String names of the integer classes
    """
    # Normalize the confusion matrix.
    cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)
    
    fig, ax = plt.subplots(figsize=(12, 12))
    im = ax.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
    ax.figure.colorbar(im, ax=ax)
    # We want to show all ticks...
    tick_marks = np.arange(len(class_names))
    ax.set(xticks=tick_marks,
           yticks=tick_marks,
           xticklabels=class_names, 
           yticklabels=class_names,
           title='Confusion matrix',
           ylabel='True label',
           xlabel='Predicted label')
    # Rotate the tick labels and set their alignment.
    plt.setp(ax.get_xticklabels(), rotation=45, ha="right", rotation_mode="anchor")
    
    # Loop over data dimensions and create text annotations.
    fmt = '.2f'
    thresh = cm.max() / 2.
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            ax.text(j, i, format(cm[i, j], fmt),
                    ha="center", va="center",
                    color="white" if cm[i, j] > thresh else "black")
    #fig.tight_layout()
    return fig
    
def plot_to_image(figure):
    """
    Converts the matplotlib plot specified by 'figure' to a PNG image and
    returns it. The supplied figure is closed and inaccessible after this call.
    """
    
    buf = io.BytesIO()
    
    # Use plt.savefig to save the plot to a PNG in memory.
    plt.savefig(buf, format='png')
    
    # Closing the figure prevents it from being displayed directly inside
    # the notebook.
    plt.close(figure)
    buf.seek(0)
    
    # Use tf.image.decode_png to convert the PNG buffer
    # to a TF image. Make sure you use 4 channels.
    image = tf.image.decode_png(buf.getvalue(), channels=4)
    
    # Use tf.expand_dims to add the batch dimension
    image = tf.expand_dims(image, 0)
    
    return image

def log_confusion_matrix(epoch, logs):
    
    # Use the model to predict the values from the test_images.
    val_pred_raw = model.predict(dataset_val.get_data_pipeline().prefetch(tf.data.AUTOTUNE))
    
    val_pred = np.argmax(val_pred_raw, axis=1)
    
    # Calculate the confusion matrix using sklearn.metrics
    cm = sklearn.metrics.confusion_matrix(val_labels, val_pred)
    
    figure = plot_confusion_matrix(cm, class_names=classes)
    cm_image = plot_to_image(figure)
    
    # Log the confusion matrix as an image summary.
    with file_writer_cm.as_default():
        tf.summary.image("Confusion Matrix", cm_image, step=epoch)

In [30]:
val_labels = []  # store true labels
for _, label_batch in tqdm.tqdm(dataset_val.get_data_pipeline()): # iterate over the validation dataset
    val_labels.extend(label_batch["target"].numpy()) # append true labels
#val_labels = np.concatenate([y for _, y in dataset_val.get_data_pipeline()], axis=0)

7351it [00:13, 542.89it/s]


In [31]:
# init tensorflow callbacks
file_writer_cm = tf.summary.create_file_writer(logdir + '/cm')
cm_callback = keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)
early_stopping = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5)
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_file_path, save_weights_only=True, save_best_only=True, monitor='val_loss', mode='min')
tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

In [ ]:
model.fit(
    dataset_train.get_data_pipeline().prefetch(tf.data.AUTOTUNE),
    epochs=epochs, 
    validation_data=dataset_val.get_data_pipeline().prefetch(tf.data.AUTOTUNE),
    callbacks=[
        early_stopping, 
        checkpoint_callback,
        tensorboard_callback,
        cm_callback
    ],
    verbose=True
)

In [ ]:
model.load_weights(os.path.join(model_path, model_name, 'training', 'checkpoints'))

In [ ]:
model.save(os.path.join(model_path, model_name, 'model'))

## Time Series Comparison

In [ ]:
import time
import datetime
import operator as op
import numpy as np
from poleno_db_interface.database.filter import AndClause, OrClause, ConditionClause, DataColumn
import poleno_db_interface.database.model.poleno_data_model as pdm
import matplotlib.pyplot as plt

#### Prepare the tf pipeline
Similar to the training and testing, we set up a tf dataset pipeline. Here we use the more flexible function `query_interface_ml.prepare_tf_dataset_from_event_filter` which allows us to define any filter. Here, we use this to get all the event from one SwisensPoleno in a defined time range.

Additionally, we filter out all events that do not comply with the minimal size and solidity conditions.

In [ ]:
from poleno_db_interface.database.filter import AndClause, ConditionClause
import operator as op
import time
from tqdm import tqdm

In [ ]:
start_date  = datetime.date(2022,1,1)
end_date    = datetime.date(2022,11,30)
device_name = "poleno-5"

filter_ = AndClause(
    ConditionClause(pdm.Event.timestamp, op.gt, time.mktime(start_date.timetuple())),
    ConditionClause(pdm.Event.timestamp, op.lt, time.mktime(end_date.timetuple())),
    ConditionClause(pdm.Event.device_id_str, op.eq, device_name),
    ConditionClause(pdm.ImageAnalysis.particleArea, op.ge, 625, "img0"),
    ConditionClause(pdm.ImageAnalysis.particleArea, op.ge, 625, "img1"),
    ConditionClause(pdm.ImageAnalysis.particleSolidity, op.ge, 0.9, "img0"),
    ConditionClause(pdm.ImageAnalysis.particleSolidity, op.ge, 0.9, "img1"),
    ConditionClause(pdm.ImageAnalysis.ImageData_id, op.eq, 0, "img0"),
    ConditionClause(pdm.ImageAnalysis.ImageData_id, op.eq, 1, "img1"),
)

timeseries_dataset = query_interface_ml.prepare_tf_dataset_from_event_filter(
    filter_=filter_,
    batch_size=batch_size,
    model_features=model_features
)
timeseries_dataset.dataset_length

In [ ]:
labels = np.array(dataset_train.labels)
d = {'event_id': [], 'pred_class': [], 'pred_certainty': []}
df = pd.DataFrame(data=d)

for id_batch, feature_batch in tqdm(timeseries_dataset.get_data_pipeline(with_id=True), total=timeseries_dataset.dataset_length/batch_size):
    # compute predictions
    preds = model.predict(feature_batch)
    # append predicted labels and certainty
    y_pred = np.argmax(preds, axis = - 1)
    ind = (np.array(range(min(len(labels), len(y_pred)))), y_pred)
    certainties = preds[ind]
    classes = labels[y_pred]
    df = df.append(
        {
            'event_id': [id_.decode() for id_ in id_batch["id"].numpy()],
            'pred_class': classes,
            'pred_certainty': certainties
        },
        ignore_index=True
    )
df = df.explode(["event_id", "pred_class", "pred_certainty"]).reset_index(drop=True)

In [ ]:
from poleno_db_interface.database.query_utils import finalize_query
from poleno_db_interface.database.query_utils import DataColumn

In [ ]:
eval_dir = os.path.join(model_path, model_name, 'eval')
os.makedirs(eval_dir, exist_ok=True)
df.to_csv(os.path.join(eval_dir, 'timeseries_all.csv'))

In [ ]:
query_interface_ml.session.rollback()

In [ ]:
columns_ = [
    DataColumn(pdm.Event.event_id_uuid),
    DataColumn(pdm.Event.timestamp),
]

df_meta = pd.DataFrame(columns=["event_id", "event_timestamp"])
for df_chunk in tqdm(np.array_split(df, 100)):
    filter_ = AndClause(
        ConditionClause(
            pdm.Event.event_id_bin, "IN", [UUID(id_).bytes for id_ in df_chunk["event_id"]]
        )
    )
    result = query_interface_ml.session.execute(
        finalize_query(
            columns=columns_,
            filter_clause=filter_
        )
    )
    df_tmp = pd.DataFrame(result, columns=["event_id", "event_timestamp"])
    df_meta = df_meta.append(df_tmp)

df_meta["event_timestamp"] = df_meta["event_timestamp"].apply(float)
df_meta["event_timestamp"] = pd.to_datetime(df_meta["event_timestamp"], unit="s")

In [ ]:
merged_df = pd.merge(df_meta, df, left_on="event_id", right_on="event_id")
merged_df = merged_df.set_index("event_timestamp")

In [ ]:
merged_df

In [ ]:
merged_df[merged_df.duplicated()]

In [ ]:
merged_df.to_csv(os.path.join(eval_dir, 'timeseries_merged_all.csv'))

In [ ]:
merged_df[merged_df["pred_certainty"]>0.9][merged_df.pred_class==label]["event_id"].resample('1d').count()

In [ ]:
frames = []
for label in labels:
    tmp_df = merged_df[(merged_df["pred_certainty"]>0.80) & (merged_df.pred_class==label)]["event_id"].resample('1D').count().to_frame()
    tmp_df["label"] = label
    frames.append(tmp_df)
all_df = pd.concat(frames)

In [ ]:
all_df = all_df.reset_index()

In [ ]:
all_df

In [ ]:
import plotly.express as ex

In [ ]:
ex.line(
    all_df,
    x = "event_timestamp",
    y = "event_id",
    color = "label"
)